In [168]:
import json
import pandas as pd
import cmath
import xml.etree.ElementTree as et
import numpy

In [115]:
def json_transform(dataframe, path):
    json_data = dataframe.to_json(orient='records')

    # Save the JSON data to a file
    with open(path, 'w') as json_file:
        json_file.write(json_data)
    

In [130]:
def reconstruct_endloc(row):
    return {"x": row['pass.endLocation.x'], "y": row['pass.endLocation.y']}



In [128]:
def reconstruct_recipient(row):
    return {"id": row['pass.recipient.id'], "name": row['pass.recipient.name'], "position": row['pass.recipient.position']}

In [111]:
def reconstruct_pass(row):
    rec = reconstruct_recipient(row)
    return {"accurate": row['pass.accurate'], 'angle': row['pass.angle'], 'height': row['pass.height'], 'length': row['pass.length'], 'recipient': rec, }

In [132]:
def reconstruct_poss_team(row):
    return {"id": row['possession.team.id'], "name": row['possession.team.name'], "formation": row['possession.team.formation']}

In [133]:
def reconstruct_poss_endloc(row):
    return {"x": row['possession.endLocation.x'], "y": row['possession.endLocation.y']}

In [134]:
def reconstruct_poss_startloc(row):
    return {"x": row['possession.startLocation.x'], "y": row['possession.startLocation.y']}

In [171]:
def reconstruct_attack(row): 
    if (pd.isna(row['possession.attack.withShot'])):
        return numpy.nan
    else:
        return {"withShot": row['possession.attack.withShot'], "withShotOnGoal": row['possession.attack.withShotOnGoal'], "withGoal": row['possession.attack.withGoal'], "flank": row['possession.attack.flank'], "withGoal": row['possession.attack.withGoal'], "xg": row['possession.attack.xg']}


In [138]:
reconstruc_list = [
    "type",
    "location",
    "team", 
    "opponentTeam",
    "player",
    "pass",
    "shot",
    "groundDuel",
    "aerialDuel",
    "infraction",
    "carry",
    "possession"
]

In [174]:
def getgoalkeeper(row):
    return {"goalkeeperActionId": row['shot.goalkeeperActionId'], "id": row['shot.goalkeeper.id'], "name": row['shot.goalkeeper.name']}

In [175]:
def getopponent(row):
    return {"id": row['groundDuel.opponent.id'], "name": row['groundDuel.opponent.name'], "position": row['groundDuel.opponent.position']}

In [176]:
def getopponent_aerial(row):
    return {"id": row['aerialDuel.opponent.id'], "name": row['aerialDuel.opponent.name'], "position": row['aerialDuel.opponent.position'], "height": row['aerialDuel.opponent.height']}

In [177]:
def getopponent_infraction(row):
    return {"id": row['infraction.opponent.id'], "name": row['infraction.opponent.name'], "position": row['infraction.opponent.position']}

In [ ]:
def reconstruct_carry_loc(row):
    return {"x": row['possession.endLocation.x'], "y": row['possession.endLocation.y']}

In [180]:
def reconstruct_cases(row, keyword): 
    match keyword:
        case "type":
            return {"primary": row['type.primary'], "secondary": row['type.secondary']}
        case "location":
            if (pd.isna(row['location.x'])):
                return numpy.nan
            return {"x": row['location.x'], "y": row['location.y']}
        case "team": 
            return {"id": row['team.id'], "name": row['team.name'], "formation": row['team.formation']}
        case "opponentTeam":
            return {"id": row['opponentTeam.id'], "name": row['opponentTeam.name'], "formation": row['opponentTeam.formation']}
        case "player": 
            return {"id": row['player.id'], "name": row['player.name'], "position": row['player.position']}
        case "pass":
            if (pd.isna(row['pass.accurate'])):
                return numpy.nan
            rec = reconstruct_recipient(row)
            endloc =reconstruct_endloc(row)
            return {"accurate": row['pass.accurate'], "angle": row['pass.angle'], "height": row['pass.height'], "length": row['pass.length'], "recipient": rec, "endLocation": endloc}
        case "possession": 
            if (pd.isna(row['possession.startLocation.x'])):
                return numpy.nan
            startloc = reconstruct_poss_startloc(row)
            endloc = reconstruct_poss_endloc(row)
            team = reconstruct_poss_team(row)
            attack = reconstruct_attack(row)
            return {"id": row['possession.id'], "duration": row['possession.duration'], "types": row['possession.types'], "eventsNumber": row['possession.types'], "eventIndex": row['possession.types'], "startLocation": startloc,"endLocation": endloc, "team": team, "attack": attack}
        case "shot": 
            if (pd.isna(row['shot.bodyPart'])):
                return numpy.nan
            keeper = getgoalkeeper(row)
            return {"bodyPart": row['shot.bodyPart'], "isGoal": row['shot.isGoal'], "onTarget": row['shot.onTarget'], "goalZone": row['shot.goalZone'], "xg": row['shot.xg'], "postShotXg": row['shot.postShotXg'], "goalkeeper": keeper}
        case "groundDuel": 
            if (pd.isna(row['groundDuel.duelType'])):
                return numpy.nan
            opponent = getopponent(row)
            return {"opponent": opponent, "duelType": row['groundDuel.duelType'], "keptPossession": row['groundDuel.keptPossession'], "progressedWithBall": row['groundDuel.progressedWithBall'], "stoppedProgress": row['groundDuel.stoppedProgress'], "recoveredPossession": row['groundDuel.recoveredPossession'], "takeOn": row['groundDuel.takeOn'], "side": row['groundDuel.side'], "relatedDuelId": row['groundDuel.relatedDuelId'] }
        case "aerialDuel":
            if (pd.isna(row['aerialDuel.opponent.name'])):
                return numpy.nan
            opponent = getopponent_aerial(row)
            return {"opponent": opponent, "firstTouch": row['aerialDuel.firstTouch'], "height": row['aerialDuel.height'], "relatedDuelId": row['aerialDuel.relatedDuelId']}
        case "infraction": 
            if (pd.isna(row['infraction.yellowCard'])):
                return numpy.nan
            opponent = getopponent_infraction(row)
            return {"yellowCard": row['infraction.yellowCard'], "redCard": row['infraction.redCard'], "type": row['infraction.type'], "opponent": opponent}
        case "carry":
            if (pd.isna(row['carry.progression'])):
                return numpy.nan
            return {"progression": row['carry.progression'], "endLocation": {"x": row['carry.endLocation.x'], "y": row['carry.endLocation.y']}}







In [113]:
path = 'data/Wyscout_Servette_FCZ_2023-06-02.json'

In [125]:
with open(path, 'r') as f:
    rd = f.read()
obj = json.loads(rd)
dfw = pd.json_normalize(obj)
dfw

,meta,events
0,[],"[{'id': 1669813357, 'matchId': 5461231, 'match..."


In [160]:
single_entry = dfw['events'].iloc[0]

expanded_table = pd.json_normalize(single_entry)
expanded_table

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,...,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.opponent,location
0,1669813357,5461231,1H,0,3,00:00:03.210,4.210517,1.669813e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1669813358,5461231,1H,0,5,00:00:05.478,6.478266,1.669813e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1669813359,5461231,1H,0,12,00:00:12.267,13.267664,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1669813427,5461231,1H,0,19,00:00:19.628,20.628473,1.669813e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1669813430,5461231,1H,0,22,00:00:22.068,23.0683,1.669813e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,1669815080,5461231,2H,93,49,01:33:49.768,5652.768946,1.669815e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1566,1669814887,5461231,2H,93,52,01:33:52.137,5655.13785,1.669815e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1567,1669814888,5461231,2H,93,53,01:33:53.359,5656.359792,1.669815e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1568,1669815081,5461231,2H,93,58,01:33:58.521,5661.521425,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
expanded_table['type'] = expanded_table.apply(lambda row: reconstruct_cases(row, "possession"), axis=1)
dl=expanded_table.replace(pd.NA, None)
dl=expanded_table.replace(numpy.nan, "heelo")
dl

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,...,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.opponent,location,type
0,1669813357,5461231,1H,0,3,00:00:03.210,4.210517,1669813358.0,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669813357.0, 'duration': '5.662448', '..."
1,1669813358,5461231,1H,0,5,00:00:05.478,6.478266,1669813359.0,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669813357.0, 'duration': '5.662448', '..."
2,1669813359,5461231,1H,0,12,00:00:12.267,13.267664,heelo,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669813357.0, 'duration': '5.662448', '..."
3,1669813427,5461231,1H,0,19,00:00:19.628,20.628473,1669813430.0,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669813427.0, 'duration': '4.0214045', ..."
4,1669813430,5461231,1H,0,22,00:00:22.068,23.0683,1669813360.0,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669813427.0, 'duration': '4.0214045', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,1669815080,5461231,2H,93,49,01:33:49.768,5652.768946,1669814887.0,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669815078.0, 'duration': '7.560258', '..."
1566,1669814887,5461231,2H,93,52,01:33:52.137,5655.13785,1669814888.0,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669814887.0, 'duration': '3.8027585', ..."
1567,1669814888,5461231,2H,93,53,01:33:53.359,5656.359792,1669815081.0,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669814887.0, 'duration': '3.8027585', ..."
1568,1669815081,5461231,2H,93,58,01:33:58.521,5661.521425,heelo,heelo,heelo,...,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,heelo,"{'id': 1669814887.0, 'duration': '3.8027585', ..."


In [ ]:
expanded_dict = expanded_table.to_dict(orient='records')
nested_structure = {
    'events': expanded_dict  # Assuming 'table' is the column where the nested structure resides
}

concatenated_result = '['
x=pd.DataFrame(nested_structure)
for index, row in x.iterrows():
   concatenated_result += str(row['events']) + ', '

concatenated_result = concatenated_result[:-2]
concatenated_result += ']'

newdata= {
    'meta': [[]],
    'events': concatenated_result
}
print(type(concatenated_result))
y=pd.DataFrame(newdata)
y



In [ ]:
json_data = y.to_json(orient='records')
file_path = 'output2.json'

# Save the JSON data to a file
with open(file_path, 'w') as json_file:
    json_file.write(json_data)